Dynamic Traveling Salesman Problem (DTSP) with RL

In [ ]:


import or_gym
from or_gym.utils import create_env
import numpy as np
print(np.version)
import matplotlib.pyplot as plt
from matplotlib import gridspec


#print(np.version.version)

env_config = {}

# Environment and RL Configuration Settings
env_name = 'TSP-v0'
# env_name = "Knapsack-v0"
env_config = {} # Change environment parameters here
 
# Register environment
env = create_env(env_name)



